In [ ]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

!conda create -n internvl python=3.9 -y

!source activate internvl

In [ ]:
!git clone https://github.com/Czi24/InternVL-20240714.git
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

!pip install flash-attn==2.3.6 --no-build-isolation
!pip install transformers==4.37.2
!pip install opencv-python termcolor yacs pyyaml scipy
!pip install deepspeed==0.13.5
!pip install pycocoevalcap tqdm

%cd /content/InternVL-20240714/internvl_chat_llava/
!pip install -e .

In [12]:
!pip install huggingface_hub protobuf
from huggingface_hub import snapshot_download
%cd /content
snapshot_download(repo_id="OpenGVLab/InternVL2-4B", local_dir="InternVL2-4B")

/content


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

modeling_phi3.py:   0%|          | 0.00/74.4k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

'/content/InternVL2-4B'

In [13]:
%cd /content/InternVL-20240714/internvl_chat_llava/
import threading
import subprocess

# start Controller
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'],
        check=True
    ),
    daemon=True
).start()

/content/InternVL-20240714/internvl_chat_llava


In [14]:
%cd /content/InternVL-20240714/internvl_chat
import threading
import subprocess

# Define the command to run the model worker
command = [
    'python3', '-m', 'internvl.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40001',
    '--worker', 'http://localhost:40001',
    '--model-path', '/content/InternVL2-4B'
]

# Run the command in a separate thread
thread = threading.Thread(
    target=lambda: subprocess.run(command, check=True, shell=False),
    daemon=True
)
thread.start()


/content/InternVL-20240714/internvl_chat


In [15]:
%cd /content/InternVL-20240714/internvl_chat_llava
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share

/content/InternVL-20240714/internvl_chat_llava
2024-07-15 12:33:45 | INFO | gradio_web_server | args: Namespace(host='0.0.0.0', port=None, controller_url='http://localhost:10000', concurrency_count=10, model_list_mode='reload', share=True, moderate=False, embed=False)
2024-07-15 12:33:45 | INFO | gradio_web_server | Models: ['InternVL2-4B']
2024-07-15 12:33:45 | INFO | gradio_web_server | Namespace(host='0.0.0.0', port=None, controller_url='http://localhost:10000', concurrency_count=10, model_list_mode='reload', share=True, moderate=False, embed=False)
2024-07-15 12:33:45 | INFO | stdout | Running on local URL:  http://0.0.0.0:7860
2024-07-15 12:33:46 | INFO | stdout | Running on public URL: https://775f0a65c060c079c5.gradio.live
2024-07-15 12:33:46 | INFO | stdout | 
2024-07-15 12:33:46 | INFO | stdout | This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
2024-07-15 12:3

In [11]:
import os
import signal

# Function to kill all threads and processes
def kill_all_processes():
    # Get the current process id
    pid = os.getpid()

    # Get the list of child process ids
    try:
        pids = subprocess.check_output(['pgrep', '-P', str(pid)])
        pids = [int(p) for p in pids.split()]
    except subprocess.CalledProcessError:
        pids = []

    # Kill all child processes
    for pid in pids:
        os.kill(pid, signal.SIGKILL)

# Kill all processes
kill_all_processes()


Exception in thread Thread-13 (<lambda>):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-2b9888d8bb26>", line 7, in <lambda>
  File "/usr/lib/python3.10/subprocess.py", line 526, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000']' died with <Signals.SIGKILL: 9>.
Exception in thread Thread-14 (<lambda>):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-9901044abea7>", line 17, in <lambda>
  File "/usr/lib/python3.10/subprocess.py", line 526, in run
  

In [ ]:
%cd /content/InternVL-20240714/internvl_chat_llava/
import threading
import subprocess

# Start Controller
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'],
        check=True
    ),
    daemon=True
).start()

# Start Gradio Web Server
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.gradio_web_server', '--controller', 'http://localhost:10000', '--model-list-mode', 'reload', '--share'],
        check=True
    ),
    daemon=True
).start()


/content/InternVL-20240714/internvl_chat_llava


In [ ]:
%cd /content/InternVL-20240714/internvl_chat
!python3 -m internvl.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40005 --worker http://localhost:40005 --model-path /content/InternVL2-2B-AWQ
